Overview

<h2> Project Set-up <h2>

In [2]:
source("sources/ggrare.R") #github library: https://rdrr.io/github/gauravsk/ranacapa/
source("sources/miseqR.R")
source("sources/qiime2R.R") #github https://github.com/jbisanz/qiime2R
source('sources/abundances.R') #github https://github.com/microbiome/microbiome

library(biomformat)
library(ape)
library(cowplot)
library(cluster)
library(data.table)
library(dplyr)
library("expss")
library(ggplot2)
library("ggpubr")
library(Hmisc)
library(otuSummary)
library(phyloseq)
library(scales)
library(tibble)
library(vegan)
library(yaml)

Warning message:
"package 'ape' was built under R version 3.6.3"
Warning message:
"package 'cowplot' was built under R version 3.6.3"

********************************************************

Note: As of version 1.0.0, cowplot does not change the

  default ggplot2 theme anymore. To recover the previous

  behavior, execute:
  theme_set(theme_cowplot())

********************************************************


Warning message:
"package 'cluster' was built under R version 3.6.3"
Warning message:
"package 'data.table' was built under R version 3.6.3"
Warning message:
"package 'dplyr' was built under R version 3.6.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:data.table':

    between, first, last


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Warning message:
"package 'expss' was built under R version 3.6.3"

Attaching package:

In [3]:
proj_dir='/Users/slsevilla/Google Drive/MyDocuments_Current/Education/George Mason University/Dissertation/Data/Aim1/'
out_dir=paste(proj_dir,'output/',sep="")
img_dir=paste(proj_dir,out_dir,'img/',sep="")

In [4]:
#Read in taxonomy file
CONTdf = read.csv('../taxonomy/Taxonomy_Complete.csv',stringsAsFactors = FALSE)

#Convert the tax file to SILVA naming with "D_5__"
CONTdf_s = CONTdf
for (i in 1:nrow(CONTdf)){
    old = as.character(CONTdf[i,"Genus"])
    new = paste("D_5__",old,sep="")
    new = gsub(" ", ".", new, fixed = TRUE)
    CONTdf_s[i,"Genus"] = new
    
    old = as.character(CONTdf[i,"Family"])
    new = paste("D_4__",old,sep="")
    new = gsub(" ", ".", new, fixed = TRUE)
    CONTdf_s[i,"Family"] = new
}

#Convert the tax file to gg naming with "f"
CONTdf_g = CONTdf
for (i in 1:nrow(CONTdf)){
    old = as.character(CONTdf[i,"Genus"])
    new = paste("g__",old,sep="")
    new = gsub(" ", ".", new, fixed = TRUE)
    CONTdf_g[i,"Genus"] = new
    
    old = as.character(CONTdf[i,"Family"])
    new = paste("f__",old,sep="")
    new = gsub(" ", ".", new, fixed = TRUE)
    CONTdf_g[i,"Family"] = new
}

<h2> Create PhyloSeq Object<h2>

In [5]:
phylo_create <- function(tax_in,tab_in){
    #Read OTUS
    otus<-read_qza(tab_in)

    #Read Greengenes taxonomy file
    taxonomy<-read_qza(tax_in)

    #Create taxonomy table
    tax_table = data.frame()
    col = 1
    row = 1

    for (i in taxonomy$data$Taxon){
        tax_list = as.list(strsplit(as.character(i),";"))
        for (i in 1:lengths(tax_list)){
            tax_table[row,col] = tax_list[[1]][i]
            col = col + 1
        }
        col = 1
        row = row + 1
    }
    rownames(tax_table)<-taxonomy$data$Feature.ID
    colnames(tax_table)<-c("Domain", "Phylum", "Class", "Order", "Family", "Genus", "Species")
    tax_convert<-as.matrix(tax_table)  

    #read metadata
    metadata<-read.table('../manifest/m_cleaned.txt',sep='\t', header=T, row.names=1, comment="")

    #Create phylo object
    phy_obj<-phyloseq(otu_table(otus$data, taxa_are_rows = T), tax_table(tax_convert), sample_data(metadata))
    print(phy_obj) 
    return(phy_obj)
}   

In [38]:
phy_ext <- function(phy_in){
    #glom to tax level, subset for only ext controls, std processing receipts
    phy_obj = tax_glom(phy_in,taxrank="Genus") #only merges genera that have the same upper taxonomy
    phy_obj = subset_samples(phy_obj,Sample.Type=="Ext_Control") #include only controls, no blanks
    return(phy_obj)
}

In [12]:
otu_extract <- function(phy_in){
    # Extract abundance matrix from the phyloseq object
    OTU1 = as(otu_table(phy_in), "matrix")
    if(taxa_are_rows(phy_in)){OTU1 <- t(OTU1)} # transpose
    OTUdf = as.data.frame(OTU1)# Coerce to data.frame
    return(OTUdf)
}

In [13]:
tax_extract <- function(phy_in){
    #extract taxonomy information into dataframe
    TAXdf = tax_table(phy_in)
    return(TAXdf)
}

In [41]:
otu_tax_clean <- function(OTU_in,TAX_in){
    #Remove zero cols
    OTUdf_clean = OTU_in[, which(colSums(OTU_in) != 0)]

    #rename col from features to genus
    for (i in colnames(OTU_in)){
        newname = paste(as.character(TAX_in[i,"Family"]),as.character(TAX_in[i,"Genus"]),sep="")
        
        #standardize gg naming
        newname = trimws(newname) #remove first space in gg naming
        newname = sub(" g__","g__",newname)
        newname = sub("\\[","",newname)
        newname = sub("\\]","",newname)
        
        if(is.na(newname)){
            next;
        } else{
            names(OTUdf_clean)[names(OTUdf_clean) == i] <- newname
        }
    }
    
    #add metadata col
    metadata<-read.table('../manifest/m_cleaned.txt',sep='\t', header=T, row.names=1, comment="")
    for (a in rownames(OTUdf_clean)){
        OTUdf_clean[a,"Subject.ID"] = metadata[a,"Subject.ID"]
        OTUdf_clean[a,"Ext.Kit"] = metadata[a,"Ext.Kit"]
        OTUdf_clean[a,"Homo.Status"] = metadata[a,"Homo.Status"]
    }
    OTUdf_clean = data.frame(OTUdf_clean,stringsAsFactors=FALSE)
    
    return(OTUdf_clean)
}

In [58]:
#Create TP,FP,FN dataframe
bench_compare <- function(OTUdf_in,CONTdf_in){
    
    F1df = OTUdf_in
    F1df$Sample_names = rownames(F1df) #create col of sample_names
    
    #For each of the controls in the cleaned df
    for (cont in unique(OTUdf_in$Subject.ID)){
        temp=filter(CONTdf_in, get(cont) !=0) #create df of taxa related to control
        
        #Create taxa list, merging family and genus - this will help avoid id errors from taxonomic differences
        #IE one taxa labeled at the genus level may be from family1 and the same taxa may also be from family2
        taxa_list=list()
        for (x in rownames(temp)){taxa_list = c(taxa_list,paste(temp[x,"Family"],temp[x,"Genus"],sep=""))}  
        
        #Create list of samples that match the control
        sample_list = filter(F1df, Subject.ID == cont)$Sample_names
        
        #If the data is not in the df (FN), then add the taxa as a new col
        for (taxa in taxa_list){ 
            if(!(taxa %in% unique(colnames(F1df)))){
                F1df[taxa[!(taxa %in% colnames(F1df))]] = 0
            } 
        }

        for (sample in sample_list){
            for (col in unique(colnames(F1df))[unique(colnames(F1df)) != "Subject.ID" & unique(colnames(F1df)) != "Sample_names"
                & unique(colnames(F1df)) != "Ext.Kit" & unique(colnames(F1df)) != "Homo.Status"]){ 
                #removes subjectID, sample_names cols from loop
                value_check = F1df[sample,col]

                if(col %in% taxa_list && value_check > 0){ #column matches expected taxa, has features
                    F1df[sample,col] = "TP"
                } else if (col %in% taxa_list && value_check == 0){ #column matches expected taxa, no features
                    F1df[sample,col] = "FN"
                } else if ((!col %in% taxa_list) && value_check > 0){ #column doesnt match expected taxa, has features
                    F1df[sample,col] = "FP"
                } else{ #column doesnt match expected taxa, has no features
                    next
                }
            }
        }
    }
    return(F1df)
}

In [64]:
#Create counts of FP, FN, TP
bench_counts <- function(OTUdf_in,F1_in,bmark_in,var_in){
    F1df_counts = subset(OTUdf_in, select=c("Subject.ID"))

    for (sample in rownames(F1df)){
        FN = rowSums(F1_in=="FN")[sample][[1]]
        FP = rowSums(F1_in=="FP")[sample][[1]]
        TP = rowSums(F1_in=="TP")[sample][[1]]

        F1df_counts[sample,"check"]= FN + TP
        F1df_counts[sample,"FN"] = FN
        F1df_counts[sample,"FP"] = FP
        F1df_counts[sample,"TP"] = TP
        F1df_counts[sample,"TAR"] = TP / (TP + FP)
        F1df_counts[sample,"TDR"] = TP / (TP + FN)

        num = F1df_counts[sample,"TAR"]*F1df_counts[sample,"TDR"]
        den = F1df_counts[sample,"TAR"]+F1df_counts[sample,"TDR"] 
        F1df_counts[sample,"F1"] = 2*(num/den)
        F1df_counts[sample,"Subject.ID"] = OTUdf_in[sample,"Subject.ID"]
        
        F1df_counts[sample,"bmark"] = bmark_in
        F1df_counts[sample,"group.type"] = var_in
    }
    return(F1df_counts)
}

In [13]:
#Not used
bench_means <- function(F1_in){
    df = with(F1_in, tapply(X = FN,       
                    INDEX = Subject.ID,   
                    FUN = mean,     
                    na.rm = TRUE))
    return(df)
}

<h3> Plotting <h3>

In [82]:
plot_violin <- function(df_in,x_lab,y_ax,y_lab,col_in){
    p = ggplot(df_in, aes(x=get(x_lab), y=get(y_ax), color=get(col_in))) + 
        geom_violin(trim=FALSE) + 
        labs(color = col_in) +
        theme(axis.text.x = element_text(angle = 90, hjust = 1),
              axis.title.x = element_blank(), 
             legend.position = "none") + 
        ylab(y_lab) +
        stat_summary(fun.data=mean_sdl, color="red")
    return (p)
}

In [228]:
plot_f1_violin <- function(F1_in,x_list,y_list,id_in,title,ftype){
    
    p_1 = plot_violin(F1_in,id_in,x_list[1],y_list[1],id_in)
    p_2 = plot_violin(F1_in,id_in,x_list[2],y_list[2],id_in)
    p_3 = plot_violin(F1_in,id_in,x_list[3],y_list[3],id_in)

    #Create count violin plots
    fig = ggarrange(p_1, p_2, p_3, 
              labels = c("A", "B", "C"),
              title = title,
              #common.legend = TRUE,
              ncol = 2, nrow = 2)
    annotate_figure(fig, #https://rpkgs.datanovia.com/ggpubr/reference/annotate_figure.html
                   top = text_grob(title, color = "blue", face = "bold", size = 14)
    )
    
    ggsave(paste(out_dir,"/img/violin_f1_",ftype,"_",title,".jpg",sep=""))
}

<h3> Relative Abunandance <h3>

In [105]:
cal_relabun <- function(df_in){
    temp = df_in
    temp$Total = rowSums(df_in[,1:(ncol(df_in)-3)]) #increased from -1 to -3
    col_list = colnames(temp[,1:(ncol(temp)-4)])
    
    for (row in rownames(temp)){
        for (col in col_list){
            val = temp[row,col]
            if(val==0){
                next
            } else{
                temp[row,col] = val/temp[row,"Total"]
            }
        }
    }
    return(temp)
}

<h2> Expected to Observed Correlation <h2>

In [176]:
cal_EO <- function(df_in,contdf_in,bench_in,title_in){
    count=1
    plot_list = list()
    EOdf_merged = data.frame()
    eo_out = data.frame()
    
    #For each of the controls in the cleaned df
    for (cont in unique(df_in$Subject.ID)){

        taxdf = create_taxadf(cont,contdf_in)
        taxa_list=taxdf$taxa.id
        
        #Create list of samples that match the control
        sample_list = filter(RAdf, Subject.ID == cont)$Sample_name
        EOdf= data.frame()
        
        for (taxa in taxa_list){
            for(sample in sample_list){
                EOdf[nrow(EOdf)+1,"Sample_name"] = sample
                EOdf[nrow(EOdf),"taxa.id"] = taxa
                EOdf[nrow(EOdf),"Subject.ID"] = cont
                EOdf[nrow(EOdf),"exp"] = taxdf[taxa,cont]
                EOdf[nrow(EOdf),"bench"] = bench_in

                if(is.null(df_in[sample,taxa])){
                    next
                } else{
                    EOdf[nrow(EOdf),"obs"] = df_in[sample,taxa]
                }
            }
        }
        EOdf_merged = rbind(EOdf_merged,EOdf) #Merge dfs
        
        #Create scatter plots
        plot_list[[count]] = plot_EO(EOdf,cont)
        count=count+1
        
        #Calc stats
        EOdf[is.na(EOdf)] = 0
        temp = cal_EO_stattest(EOdf,cont,bench_in)
        eo_out = rbind(eo_out,temp) #merge df       
    }
    
    #print plots
    if(bench_in=="MagAttract PowerSoil DNA Kit"){ #four controls
        fig = ggarrange(plot_list[[1]], plot_list[[2]], plot_list[[3]], plot_list[[4]],
                        labels = c("A", "B", "C", "D"), #edit number of controls
                        ncol = 2, nrow = 3)        
    } else if(bench_in=="QIAamp with Modifications" | bench_in=="DSP Virus" | bench_in == "96 MagBead DNA Extraction Kit"){ 
        #two controls
        fig = ggarrange(plot_list[[1]], plot_list[[2]], 
                        labels = c("A", "B"), #edit number of controls
                        ncol = 2, nrow = 3)
    } else{
        fig = ggarrange(plot_list[[1]], plot_list[[2]], plot_list[[3]],
                        labels = c("A", "B", "C"), #edit number of controls
                        ncol = 2, nrow = 3)
    }

    annotate_figure(fig, top = text_grob(title_in, color = "blue", face = "bold", size = 14))
    ggsave(paste(out_dir,"/img/scatter_extstd_",bench_in,".jpg",sep=""))

    return(eo_out)
}

In [111]:
create_taxadf <- function(cont,contdf_in){
    taxdf=filter(contdf_in, get(cont) !=0) #create df of taxa related to control
    taxdf$taxa.id=paste(taxdf$Family,taxdf$Genus,sep="") #create fam/genus col
    taxdf = taxdf[,c("taxa.id",cont),drop=FALSE] #remove all cols but the merged and control col
    taxdf = aggregate(get(cont) ~ taxa.id, data = taxdf, sum) #collapse df for any dups    
    names(taxdf)[2] = cont
    rownames(taxdf) = taxdf$taxa.id
    return(taxdf)
}

In [112]:
plot_EO <- function(df_in,cont_in){
    my_data=subset(df_in,Subject.ID==cont_in)

    p = ggscatter(my_data, x = "obs", y = "exp", 
              add = "reg.line", conf.int = TRUE, 
              cor.coef = TRUE, cor.method = "pearson",
              xlab = "Expected Rel Abunance (%)", ylab = "Observed Rel Abun (%)") +
        ggtitle(paste("Scatter plot of Rel Abun:",cont_in,sep=""))
    return(p)
}

In [114]:
cal_EO_stattest <- function(df_in,cont_in,bench_in){
    my_data=subset(df_in,Subject.ID==cont_in)
    df_out=data.frame()
    
    df_out[nrow(df_out)+1,"bmark"] = bench_in
    df_out[nrow(df_out),"cont"] = cont_in
    df_out[nrow(df_out),"shapiro"] = shapiro.test(my_data$obs)$p.value
    df_out[nrow(df_out),"spear.p"] = cor.test(my_data$obs,my_data$exp,method = "spearman")$p.value
    df_out[nrow(df_out),"spear.rho"] = cor.test(my_data$obs,my_data$exp,method = "spearman")$estimate[[1]]

    #-1 indicates a strong negative correlation : this means that every time x increases, y decreases (left panel figure)
    #0 means that there is no association between the two variables (x and y) (middle panel figure)
    #1 indicates a strong positive correlation : this means that y increases with x (right panel figure)
    return(df_out)
}

<h2>Dissimilarity <h2>

In [133]:
#Create averaged relative abundance df
create_av_RAdf <- function(RAdf_in,bench_in,var_in){
    tmp = aggregate(RAdf_in[,1:(ncol(RAdf_in)-5)], list(RAdf_in$Subject.ID), mean) #increase from 3 to 5
    tmp$bmark=bench_in
    tmp$group.type = var_in
    return(tmp)
}

<h3> Merged metrics <h3>

In [187]:
#Add expected values to RA average dfs
RAdf_addcontrols<-function(cont_in,df_in){
    
    tmp=get(df_in)
    cont_list = unique(tmp$Group.1)
    cont_leng = length(cont_list) #chnaged to Group.1
    
    for(i in 1:cont_leng){
        cont = as.character(cont_list[i])
        taxadf = create_taxadf(cont,get(cont_in))
        tmp_count = nrow(tmp)+1
        
        for(tx_row in 1:nrow(taxadf)){
            tmp[tmp_count,"Group.1"]=cont
            tmp[tmp_count,taxadf[tx_row,"taxa.id"]] = taxadf[tx_row,cont]
            tmp[tmp_count,"bmark"]="expected"
            tmp[tmp_count,"group.type"]=tmp[i,"group.type"]
        }
        
    }
    tmp[is.na(tmp)] = 0 
    return(tmp)
}

In [158]:
calc_dist <- function(RA_in,CONTdf_in){
    rownames(RA_in)=paste(RA_in$Group.1,RA_in$bmark,sep="-")
    RA_in = subset(RA_in, select = -c(group.type,Group.1,bmark))

    taxa_list<-unique(paste(CONTdf_in$Family,CONTdf_in$Genus,sep=""))
    df_clean = RA_in[,colnames(RA_in) %in% taxa_list] #remove cols not expected

    df_out = vegdist(df_clean,method="bray")   
    return(df_out)
}

In [216]:
plot_dist_cluster <- function(df_in,ref_in){
    fname = paste(out_dir,"/img/hclust_summary_bc_allgroups_",ref_in,".png",sep="")
    png(fname,width = 900, height = 600)
    p = plot(hclust(df_in), cex = .9, main = "All Groups - Bray-Curtis Dissimilarity",
             xlab="Bioinformatic Variables", sub="")
    dev.off()
}

In [217]:
plot_dist_violin <- function(df_in,ref_in){
    df_in = as.data.frame(as.matrix(df_in))
    
    df_plot=data.frame()
    for (row in rownames(df_in)){
        splits = strsplit(row,"-")
        if(splits[[1]][2]=="expected"){
            next
        }else{
            df_plot[nrow(df_plot)+1,"dist"]=df_in[row,paste(splits[[1]][1],"expected",sep="-")]
            df_plot[nrow(df_plot),"name"]=splits[[1]][2]
        }
    }

    p = ggplot(df_plot, aes(x=name, y=dist, color=name)) + 
            geom_violin(trim=FALSE) + 
            labs(color = df_plot$name) +
            theme(axis.text.x = element_text(angle = 90, hjust = 1),
                  axis.title.x = element_blank(), 
                 legend.position = "none") + 
            xlab("Benchmark ID") +
            ylab("Bray-Curtis Dissimilarity") +
            stat_summary(fun.data=mean_sdl, color="red")
    
    
    fig = ggarrange(p, 
              #labels = c("A"),
              #top = as.character(title_in),
              #common.legend = TRUE#,
              ncol = 1, nrow = 1)
    annotate_figure(fig, top = text_grob("Bray-Curtis Dissimilarity", color = "blue", face = "bold", size = 14)
    )
    ggsave(paste(out_dir,"/img/violin_bc_summary_allgroups_",ref_in,".jpg",sep=""))
    return(df_plot)

}

<h2> Statistics <h2>

In [234]:
tstat_paired_f1 <- function(f1_in,subgroup_in){
    
    df_out = data.frame()
    
    cal_list = c("TAR","TDR","F1")
    
    for (cal in cal_list){
    
        comb_list = combn(unique(f1_in[,subgroup_in]),2)
        count=1
        
        for (i in 1:(length(comb_list)/2)){
            
            tstatdf=data.frame()
            var1 = as.character(comb_list[count])
            var2 = as.character(comb_list[count+1])
            count=count+2

            df = f1_in %>%
                filter(f1_in[,subgroup_in] == var1 | f1_in[,subgroup_in] == var2) %>%
                select(subgroup_in, cal)
            
            tmp = t.test(get(cal) ~ get(subgroup_in), data = df)
            tstatdf[nrow(tstatdf)+1,"level"] = cal
            tstatdf[nrow(tstatdf),"comparison"] = paste(var1,var2,sep="_")
            tstatdf[nrow(tstatdf),"pval"] = tmp$p.value
            if(tmp$p.value<0.05){
                tstatdf[nrow(tstatdf),"sig"] = "sig"
            } else{
                tstatdf[nrow(tstatdf),"sig"] = "ns"
            }
            

            df_out = bind_rows(df_out,tstatdf)
        }
    }
        
    return(df_out)
}

In [224]:
adonis_bc <- function(df_in){

    df_out=data.frame()
    
    st = adonis(df_in[,1] ~ name,
                data=df_in, permutations = 99, method="bray")
    df_out[nrow(df_out)+1,"comparison"] = "bray-curtis"
    df_out[nrow(df_out),"pval"] = as.numeric(st$aov.tab$"Pr(>F)"[1])
        
    if(df_out[nrow(df_out),"pval"]<0.05){
        df_out[nrow(df_out),"sig"] = "sig"
    } else{
        df_out[nrow(df_out),"sig"] = "ns"
    }   
    
    dist_s = vegdist(subset(dissdf_s,select = -c(bmark,Group.1,group.type)),na.rm=TRUE)

        
    df_out[nrow(df_out)+1,"homo_s"] = anova(betadisper(dist_s,dissdf_s$bmark))$"Pr(>F)"[1]
    
    return(df_out)
}

<h2> Run Production Code <h2>

In [150]:
#Create the phyloseq object
tax_file = paste(out_dir,'taxonomic_classification/classify-sklearn_silva-132-99-nb-classifier.qza',sep="")
table_file = paste(out_dir,'denoising/feature_tables/merged_filtered.qza',sep="")
phy_obj = phylo_create(tax_file,table_file)  

#Create glomed, subsetted obj
std_obj = phy_ext(phy_obj)

#Rarefy samples
phy_r = rarefy_even_depth(std_obj, rngseed=1, sample.size=20000, replace=F) 

#Create OTUdf
OTUdf = otu_extract(phy_r)

#Create TAXdf
TAXdf = tax_extract(phy_r)

#Clean df's
OTUdf_clean = otu_tax_clean(OTUdf,TAXdf)

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 1818 taxa and 351 samples ]
sample_data() Sample Data:       [ 351 samples by 24 sample variables ]
tax_table()   Taxonomy Table:    [ 1818 taxa by 7 taxonomic ranks ]


`set.seed(1)` was used to initialize repeatable random subsampling.

Please record this for your records so others can reproduce.

Try `set.seed(1); .Random.seed` for the full vector

...

40 samples removedbecause they contained fewer reads than `sample.size`.

Up to first five removed samples are: 


SC249366-PC04925-D-01SC249406SC249422-PC04925-D-03SC249431SC253191

...

56OTUs were removed because they are no longer 
present in any sample after random subsampling


...



<h3> Standard Processing <h3>

In [168]:
#List of all variables to test
test_list = unique(sample_data(std_obj)$Ext.Kit)
test_list
#test_list = test_list[1]

[1] DSP Virus                      96 MagBead DNA Extraction Kit 
[3] MagAttract PowerSoil DNA Kit   MagAttract PowerMicrobiome Kit
[5] QIAamp with Modifications      DNeasy PowerSoil Pro kit      
[7] MagMax Microbiome Ultra Kit   
7 Levels: 96 MagBead DNA Extraction Kit DNeasy PowerSoil Pro kit ... QIAamp with Modifications

In [173]:
#Create a merged dfs for all variables
mergedf_f1stats = data.frame()
EOdf_stats_merged = data.frame()
dissdf_s = data.frame()
dissdf_g = data.frame()

In [174]:
for (variable in test_list){
    
    #Subset dataframe
    OTUdf_clean2 = subset(OTUdf_clean,Ext.Kit==variable & Homo.Status=="Standard" & Subject.ID!="Water")
    
    #Create TP,FP,FN df
    CONTdf_use = CONTdf_s
    F1df = bench_compare(OTUdf_clean2, CONTdf_use)
    
    #Create TP,FP,FN counts    
    bench_key = "standard"
    F1df_counts = bench_counts(OTUdf_clean2,F1df,variable, bench_key)#swapped variable and benchkey from original
    temp = F1df_counts
    temp$sample_name = rownames(temp)
    rownames(temp) = c()
    mergedf_f1stats = rbind(mergedf_f1stats,temp)

    #Create violin plots for each control - counts
    bench_key = variable
    x_list = c("FN","TP","FP")
    y_labs = c("False negative","True positive", "False positive")
    plot_f1_violin(F1df_counts,x_list,y_labs,"Subject.ID",bench_key,"counts_extstd")
        
    #Create violin plots for each control - summary
    bench_key = variable
    x_list = c("TAR","TDR","F1")
    y_labs = c("Taxonmic Accuracy Rate","Taxonmic Detection Rate", "F-Measure")
    plot_f1_violin(F1df_counts,x_list,y_labs,"Subject.ID",bench_key,"summary_extstd")
    
    #calculate relative abundance
    RAdf = cal_relabun(OTUdf_clean2)
    RAdf$Sample_name = rownames(RAdf)
    
    #Create scatter plots
    #Determine expected to observed scatter plots, normality, spearman coeff 
    title = paste("Scatterplots of controls: ", variable,sep="")
    EOdf_stats = cal_EO(RAdf,CONTdf_use,variable,title)    
    #EOdf_stats_merged = rbind(EOdf_stats_merged,EOdf_stats) 
    
    #Create averaged OTU df
    bench_key = "std"
    dissdf = create_av_RAdf(RAdf,variable,bench_key) #swap order of bench and variable
    dissdf_s = bind_rows(dissdf_s,dissdf)
    
    #save files
    #write.csv(OTUdf_clean,paste(out_dir,"bench_files/OTUcounts_",bench_key,".csv",sep=""))
    #write.csv(F1df,paste(out_dir,"/bench_files/F1counts_",bench_key,".csv",sep=""))
    write.csv(F1df_counts,paste(out_dir,"bench_files/F1stats_extstd_",variable,".csv",sep=""))#swap bench for variable
}

Warning message in as_grob.default(plot):
"Cannot convert object of class character into a grob."
Saving 6.67 x 6.67 in image

Warning message in as_grob.default(plot):
"Cannot convert object of class character into a grob."
Saving 6.67 x 6.67 in image



             Sample_name                                          taxa.id
1  SC249362-PC07578-H-08                    D_4__BacillaceaeD_5__Bacillus
2  SC249368-PC07578-C-10                    D_4__BacillaceaeD_5__Bacillus
3  SC249362-PC04924-E-01                    D_4__BacillaceaeD_5__Bacillus
4  SC249368-PC04924-B-02                    D_4__BacillaceaeD_5__Bacillus
5               SC253844                    D_4__BacillaceaeD_5__Bacillus
6               SC253854                    D_4__BacillaceaeD_5__Bacillus
7  SC249362-PC07578-H-08 D_4__EnterobacteriaceaeD_5__Escherichia.Shigella
8  SC249368-PC07578-C-10 D_4__EnterobacteriaceaeD_5__Escherichia.Shigella
9  SC249362-PC04924-E-01 D_4__EnterobacteriaceaeD_5__Escherichia.Shigella
10 SC249368-PC04924-B-02 D_4__EnterobacteriaceaeD_5__Escherichia.Shigella
11              SC253844 D_4__EnterobacteriaceaeD_5__Escherichia.Shigella
12              SC253854 D_4__EnterobacteriaceaeD_5__Escherichia.Shigella
13 SC249362-PC07578-H-08           D_4

Warning message in cor.test.default(my_data$obs, my_data$exp, method = "spearman"):
"Cannot compute exact p-value with ties"
Warning message in cor.test.default(my_data$obs, my_data$exp, method = "spearman"):
"Cannot compute exact p-value with ties"


   Sample_name                                          taxa.id Subject.ID
1     SC249363              D_4__BacteroidaceaeD_5__Bacteroides    DZ35322
2     SC253848              D_4__BacteroidaceaeD_5__Bacteroides    DZ35322
3     SC253851              D_4__BacteroidaceaeD_5__Bacteroides    DZ35322
4     SC249363      D_4__BifidobacteriaceaeD_5__Bifidobacterium    DZ35322
5     SC253848      D_4__BifidobacteriaceaeD_5__Bifidobacterium    DZ35322
6     SC253851      D_4__BifidobacteriaceaeD_5__Bifidobacterium    DZ35322
7     SC249363             D_4__BukerholderiaceaeD_5__Ralstonia    DZ35322
8     SC253848             D_4__BukerholderiaceaeD_5__Ralstonia    DZ35322
9     SC253851             D_4__BukerholderiaceaeD_5__Ralstonia    DZ35322
10    SC249363           D_4__CoriobacteriaceaeD_5__Collinsella    DZ35322
11    SC253848           D_4__CoriobacteriaceaeD_5__Collinsella    DZ35322
12    SC253851           D_4__CoriobacteriaceaeD_5__Collinsella    DZ35322
13    SC249363           

Warning message in cor.test.default(my_data$obs, my_data$exp, method = "spearman"):
"Cannot compute exact p-value with ties"
Warning message in cor.test.default(my_data$obs, my_data$exp, method = "spearman"):
"Cannot compute exact p-value with ties"
`geom_smooth()` using formula 'y ~ x'

Warning message:
"Removed 6 rows containing non-finite values (stat_smooth)."
Warning message:
"Removed 6 rows containing non-finite values (stat_cor)."
Warning message:
"Removed 6 rows containing missing values (geom_point)."
`geom_smooth()` using formula 'y ~ x'

Warning message:
"Removed 15 rows containing non-finite values (stat_smooth)."
Warning message:
"Removed 15 rows containing non-finite values (stat_cor)."
Warning message:
"Removed 15 rows containing missing values (geom_point)."
Saving 6.67 x 6.67 in image

Warning message in as_grob.default(plot):
"Cannot convert object of class character into a grob."
Saving 6.67 x 6.67 in image

Warning message in as_grob.default(plot):
"Cannot convert o

...

Warning message in cor.test.default(my_data$obs, my_data$exp, method = "spearman"):
"Cannot compute exact p-value with ties"
Warning message in cor.test.default(my_data$obs, my_data$exp, method = "spearman"):
"Cannot compute exact p-value with ties"
`geom_smooth()` using formula 'y ~ x'

Warning message:
"Removed 10 rows containing non-finite values (stat_smooth)."
Warning message:
"Removed 10 rows containing non-finite values (stat_cor)."
Warning message:
"Removed 10 rows containing missing values (geom_point)."
`geom_smooth()` using formula 'y ~ x'

Warning message:
"Removed 1 rows containing non-finite values (stat_smooth)."
Warning message:
"Removed 1 rows containing non-finite values (stat_cor)."
Warning message:
"Removed 1 rows containing missing values (geom_point)."
`geom_smooth()` using formula 'y ~ x'

Warning message:
"Removed 1 rows containing non-finite values (stat_smooth)."
Warning message:
"Removed 1 rows containing non-finite values (stat_cor)."
Warning message:
"Remov

   Sample_name                                        taxa.id Subject.ID
1     SC553025                  D_4__BacillaceaeD_5__Bacillus    DZ35316
2     SC553025    D_4__BifidobacteriaceaeD_5__Bifidobacterium    DZ35316
3     SC553025       D_4__BrevibacteriaceaeD_5__Stomatococcus    DZ35316
4     SC553025     D_4__CampylobacteriaceaeD_5__Campylobacter    DZ35316
5     SC553025         D_4__CoriobacteriaceaeD_5__Eggerthella    DZ35316
6     SC553025             D_4__CoriobacteriaceaeD_5__Slackia    DZ35316
7     SC553025         D_4__EnterobacteriaceaeD_5__Klebsiella    DZ35316
8     SC553025          D_4__EubacteriaceaeD_5__Mogibacterium    DZ35316
9     SC553025      D_4__FlavobacteriaceaeD_5__Capnocytophaga    DZ35316
10    SC553025        D_4__FusobacteriaceaeD_5__Fusobacterium    DZ35316
11    SC553025         D_4__LeptotrichiaceaeD_5__Leptotrichia    DZ35316
12    SC553025            D_4__LeuconostocaceaeD_5__Weissella    DZ35316
13    SC553025               D_4__NeisseriaceaeD_5_

Warning message in cor.test.default(my_data$obs, my_data$exp, method = "spearman"):
"Cannot compute exact p-value with ties"
Warning message in cor.test.default(my_data$obs, my_data$exp, method = "spearman"):
"Cannot compute exact p-value with ties"


  Sample_name                                          taxa.id Subject.ID   exp
1    SC553030                    D_4__BacillaceaeD_5__Bacillus      D6300 0.174
2    SC553030 D_4__EnterobacteriaceaeD_5__Escherichia.Shigella      D6300 0.101
3    SC553030           D_4__EnterobacteriaceaeD_5__Salmonella      D6300 0.104
4    SC553030            D_4__EnterococcaceaeD_5__Enterococcus      D6300 0.099
5    SC553030          D_4__LactobacillaceaeD_5__Lactobacillus      D6300 0.184
6    SC553030                   D_4__ListeriaceaeD_5__Listeria      D6300 0.141
7    SC553030            D_4__PseudomonadaceaeD_5__Pseudomonas      D6300 0.042
8    SC553030        D_4__StaphylococcaceaeD_5__Staphylococcus      D6300 0.155
                        bench     obs
1 MagMax Microbiome Ultra Kit 0.11235
2 MagMax Microbiome Ultra Kit 0.33090
3 MagMax Microbiome Ultra Kit 0.00000
4 MagMax Microbiome Ultra Kit 0.16395
5 MagMax Microbiome Ultra Kit 0.07105
6 MagMax Microbiome Ultra Kit 0.06160
7 MagMax Micro

`geom_smooth()` using formula 'y ~ x'

Warning message:
"Removed 10 rows containing non-finite values (stat_smooth)."
Warning message:
"Removed 10 rows containing non-finite values (stat_cor)."
Warning message:
"Removed 10 rows containing missing values (geom_point)."
`geom_smooth()` using formula 'y ~ x'

Warning message:
"Removed 1 rows containing non-finite values (stat_smooth)."
Warning message:
"Removed 1 rows containing non-finite values (stat_cor)."
Warning message:
"Removed 1 rows containing missing values (geom_point)."
`geom_smooth()` using formula 'y ~ x'

Warning message:
"Removed 1 rows containing non-finite values (stat_smooth)."
Warning message:
"Removed 1 rows containing non-finite values (stat_cor)."
Warning message:
"Removed 1 rows containing missing values (geom_point)."
Saving 6.67 x 6.67 in image



In [175]:
write.csv(EOdf_stats_merged,paste(out_dir,"bench_files/EO_stats_summary_extstd.csv",sep=""))
write.csv(mergedf_f1stats,paste(out_dir,"bench_files/F1_stats_summary_extstd.csv",sep=""))

<h4> Benchmarked Assessment<h4>

In [219]:
Distdf_merged=data.frame()
#Silva
if(length(test_list)>1){
    RAdf_oe = RAdf_addcontrols("CONTdf_s","dissdf_s")
    RAdf_clean = calc_dist(RAdf_oe,CONTdf_s)
    plot_dist_cluster(RAdf_clean,"extstd")
    Distdf = plot_dist_violin(RAdf_clean,"extstd")
} 
Distdf_merged = bind_rows(Distdf_merged,Distdf)

Saving 6.67 x 6.67 in image



In [225]:
adonis_stats = adonis_bc(Distdf_merged)
adonis_stats

Warning message in betadisper(dist_s, dissdf_s$bmark):
"some squared distances are negative and changed to zero"


,comparison,pval,sig,homo_s
,<chr>,<dbl>,<chr>,<dbl>
1,bray-curtis,0.78,ns,NA
2,NA,NA,NA,0.9781328


In [232]:
#Overall violin plots
x_list = c("TAR","TDR","F1")
y_labs = c("Taxonmic Accuracy Rate","Taxonmic Detection Rate", "F-Measure")

#Create violin plots collapsed to the group.type (merges all ref dbs, and tax class methods)
plot_f1_violin(mergedf_f1stats,x_list,y_labs,"bmark","Ext Protocols Ext Std","summary")

#Create violin plots collapsed to the tax.type (merges all ref dbs, and grouping methods)
#plot_f1_violin(mergedf_f1stats,x_list,y_labs,"tax.type","Tax Types","summary")

#Create violin plots collapsed to the tax.ref (merges all tax class methods and grouping methods)
#plot_f1_violin(mergedf_f1stats,x_list,y_labs,"tax.ref","Tax Ref Types","summary")

#Create violin plots for all binf variations (merges all variables)
#plot_f1_violin(mergedf_f1stats,x_list,y_labs,"bmark","All Grouping and Assignment Variables","summary")

Warning message in as_grob.default(plot):
"Cannot convert object of class character into a grob."
Saving 6.67 x 6.67 in image



In [235]:
#T-tests
tstatdf_merged=data.frame()
variable_list = c("bmark")

for (variable in variable_list){
    tstatdf = tstat_paired_f1(mergedf_f1stats,variable)
    tstatdf_merged=bind_rows(tstatdf_merged,tstatdf)  
}
write.csv(tstatdf_merged,paste(out_dir,"bench_files/ttest_stats_extstd.csv",sep=""))

Note: Using an external vector in selections is ambiguous.
i Use `all_of(subgroup_in)` instead of `subgroup_in` to silence this message.
i See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.

Note: Using an external vector in selections is ambiguous.
i Use `all_of(cal)` instead of `cal` to silence this message.
i See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.



<h3> Blank assessment <h3>